Notebook to show what one could do with the ouput CSV file of features-from-dlc script.
This uses long spontaneous recordings of open field.
The CSV file used here should contain the following columns :
- time
- speed
- theta_body
- theta_neck
- xbody
- ybody
- trial
- trialID
- condition
- animal

In [1]:
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.collections import LineCollection

%matplotlib widget

In [2]:
# full path to the CSV file generated by features-from-dlc
file = ("/path/to/output/features.csv")

# openfield dimensions in mm (width, height)
of_x = (30, 680)
of_y = (20, 620)

# speed threshold (cm/s)
speed_threshold = 60

# numeric features
features = ['speed', 'theta_body', 'theta_neck', 'xbody', 'ybody']

In [3]:
# some utility functions
def colored_line(x, y, c, ax, **lc_kwargs):
    """
    https://matplotlib.org/stable/gallery/lines_bars_and_markers/multicolored_line.html
    Plot a line with a color specified along the line by a third value.

    It does this by creating a collection of line segments. Each line segment is
    made up of two straight lines each connecting the current (x, y) point to the
    midpoints of the lines connecting the current point with its two neighbors.
    This creates a smooth line with no gaps between the line segments.

    Parameters
    ----------
    x, y : array-like
        The horizontal and vertical coordinates of the data points.
    c : array-like
        The color values, which should be the same size as x and y.
    ax : Axes
        Axis object on which to plot the colored line.
    **lc_kwargs
        Any additional arguments to pass to matplotlib.collections.LineCollection
        constructor. This should not include the array keyword argument because
        that is set to the color argument. If provided, it will be overridden.

    Returns
    -------
    matplotlib.collections.LineCollection
        The generated line collection representing the colored line.
    """
    if "array" in lc_kwargs:
        warnings.warn('The provided "array" keyword argument will be overridden')

    # Default the capstyle to butt so that the line segments smoothly line up
    default_kwargs = {"capstyle": "butt"}
    default_kwargs.update(lc_kwargs)

    # Compute the midpoints of the line segments. Include the first and last points
    # twice so we don't need any special syntax later to handle them.
    x = np.asarray(x)
    y = np.asarray(y)
    x_midpts = np.hstack((x[0], 0.5 * (x[1:] + x[:-1]), x[-1]))
    y_midpts = np.hstack((y[0], 0.5 * (y[1:] + y[:-1]), y[-1]))

    # Determine the start, middle, and end coordinate pair of each line segment.
    # Use the reshape to add an extra dimension so each pair of points is in its
    # own list. Then concatenate them to create:
    # [
    #   [(x1_start, y1_start), (x1_mid, y1_mid), (x1_end, y1_end)],
    #   [(x2_start, y2_start), (x2_mid, y2_mid), (x2_end, y2_end)],
    #   ...
    # ]
    coord_start = np.column_stack((x_midpts[:-1], y_midpts[:-1]))[:, np.newaxis, :]
    coord_mid = np.column_stack((x, y))[:, np.newaxis, :]
    coord_end = np.column_stack((x_midpts[1:], y_midpts[1:]))[:, np.newaxis, :]
    segments = np.concatenate((coord_start, coord_mid, coord_end), axis=1)

    lc = LineCollection(segments, **default_kwargs)
    lc.set_array(c)  # set the colors of each segment

    return ax.add_collection(lc)

In [ ]:
# read file
df = pd.read_csv(file)
display(df.head())

In [5]:
# get some information about the dataset
ntrials = len(df["trialID"].unique())  # number of trials
duration = df["time"].max() - df["time"].min()  # duration of the video
ndelays = len(df["time"].unique())  # number of time steps

In [6]:
# some data cleaning
# find where speed is aberrant
where_speed = df["speed"] > speed_threshold
# find where mouse is outside the openfield
where_outx = (df["xbody"] < of_x[0]) | (df["xbody"] > of_x[1])
where_outy = (df["ybody"] < of_y[0]) | (df["ybody"] > of_y[1])
where_nan = where_speed + where_outx + where_outy

# replace those values with nan
df.loc[where_nan, features] = np.nan
# interpolate
df[features] = df[features].interpolate(method="linear")

# body angle is cumulated, so we get the signed quadrant angle
df["theta_body"] = np.deg2rad(df["theta_body"])
df["theta_body"] = np.rad2deg(np.arctan2(np.sin(df["theta_body"]), np.cos(df["theta_body"])))

# groupby to extract trial data
df_trial = df.groupby("trialID")

In [ ]:
# 2D trajectories
fig, axs = plt.subplots(nrows=2, ncols=int(np.ceil(ntrials / 2)))
axs = axs.ravel()
for idx, data in enumerate(df_trial[["time", "xbody", "ybody"]]):
    name = data[0]
    t = data[1]["time"]
    x = data[1]["xbody"]
    y = data[1]["ybody"]

    lines = colored_line(x, y, t, axs[idx], linewidth=2, cmap="plasma")
    axs[idx].axis("equal")
    axs[idx].set_xlim((0, of_x[1]))
    axs[idx].set_ylim((0, of_y[1]))
    axs[idx].set_xticks(())
    axs[idx].set_yticks(())
    axs[idx].set_title(name)
    if idx == ntrials - 1:
        plt.colorbar(lines, label="time (s)")


In [ ]:
# presence heatmap
nbins = 4
fig, axs = plt.subplots(nrows=2, ncols=int(np.ceil(ntrials / 2)))
axs = axs.ravel()
for idx, data in enumerate(df_trial[["xbody", "ybody"]]):
    name = data[0]
    x = data[1]["xbody"]
    y = data[1]["ybody"]

    axs[idx] = sns.kdeplot(x=x, y=y, ax=axs[idx], fill=True, cmap="plasma")
    axs[idx].axis("equal")
    axs[idx].set_xlim((0, of_x[1]))
    axs[idx].set_ylim((0, of_y[1]))
    axs[idx].set_xticks(())
    axs[idx].set_yticks(())
    axs[idx].set_xlabel("")
    axs[idx].set_ylabel("")
    axs[idx].set_title(name)

In [ ]:
# compute MSD
fig, ax = plt.subplots()
for idx, data in enumerate(df_trial[["time", "xbody", "ybody"]]):
    name = data[0]
    dt = np.mean(np.diff(data[1]["time"]))
    delays = np.linspace(0, ndelays * dt, ndelays)
    xy = np.stack((data[1]["xbody"].to_numpy(), data[1]["ybody"].to_numpy()), axis=1)
    msd = np.zeros((ndelays,))
    err = np.zeros((ndelays,))

    for i in range(1, ndelays):
        # displacement for each delay
        d = np.sum((xy[i:] - xy[:-i])**2, axis=1)
        msd[i] = np.mean(d)
        err[i] = np.std(d) / np.sqrt(len(d))
    
    plt.plot(delays, msd, label=name)
    plt.fill_between(delays, msd - err, msd + err, alpha=0.5)

plt.xlabel("delays (s)")
plt.ylabel("msd (mm²)")
plt.legend()


In [ ]:
# compute MSR for neck
fig, ax = plt.subplots()
for idx, data in enumerate(df_trial[["time", "theta_neck"]]):
    name = data[0]
    dt = np.mean(np.diff(data[1]["time"]))
    delays = np.linspace(0, ndelays * dt, ndelays)
    x = data[1]["theta_neck"].to_numpy()
    msr = np.zeros((ndelays,))
    errr = np.zeros((ndelays,))

    for i in range(1, ndelays):
        # angular displacement for each delay
        d = (x[i:] - x[:-i])**2
        msr[i] = np.mean(d)
        errr[i] = np.std(d) / np.sqrt(len(d))
    
    plt.plot(delays, msr, label=name)
    plt.fill_between(delays, msr - errr, msr + errr, alpha=0.5)

plt.xlabel("delays (s)")
plt.ylabel("msr (neck) (deg²)")
plt.legend()

In [ ]:
# compute MSR for body
fig, ax = plt.subplots()
for idx, data in enumerate(df_trial[["time", "theta_body"]]):
    name = data[0]
    dt = np.mean(np.diff(data[1]["time"]))
    delays = np.linspace(0, ndelays * dt, ndelays)
    x = data[1]["theta_body"].to_numpy()
    msr = np.zeros((ndelays,))
    errr = np.zeros((ndelays,))

    for i in range(1, ndelays):
        # angular displacement for each delay
        d = (x[i:] - x[:-i])**2
        msr[i] = np.mean(d)
        errr[i] = np.std(d) / np.sqrt(len(d))
    
    plt.plot(delays, msr, label=name)
    plt.fill_between(delays, msr - errr, msr + errr, alpha=0.5)

plt.xlabel("delays (s)")
plt.ylabel("msr (body) (deg²)")
plt.legend()